# Imports

In [1]:
import pandas as pd
import tqdm as notebook_tqdm
from tqdm import tqdm
from datasets import load_dataset
import datetime
import re

/Users/husain/miniforge3/envs/heron_law/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Functions

In [2]:
def remove_translated_cases(df, citation_col='citation', lang_col='language', lang_primary='en', lang_secondary='fr'):
    """
    Removes rows in the secondary language (e.g., French) that are translations of cases already 
    present in the primary language (e.g., English), based on normalized court citations.

    Parameters:
    -----------
    df : pd.DataFrame
        The DataFrame containing legal case data.
    citation_col : str, optional
        The name of the column containing case citations. Default is 'citation'.
    lang_col : str, optional
        The name of the column containing language information. Default is 'language'.
    lang_primary : str, optional
        The language code to be considered as the primary version (e.g., 'en'). Default is 'en'.
    lang_secondary : str, optional
        The language code to be considered as the translated version to remove (e.g., 'fr'). Default is 'fr'.

    Returns:
    --------
    pd.DataFrame
        A filtered DataFrame with translated cases removed when the same case exists in the primary language.
    """
    court_acronyms = ['FC', 'CF'] 
    pattern = r'\b(' + '|'.join(court_acronyms) + r')\b'

    def normalize(citation):
        return re.sub(pattern, 'COURT', citation)

    df = df.copy()
    df['normalized_citation'] = df[citation_col].apply(normalize)

    primary_citations = set(df[df[lang_col] == lang_primary]['normalized_citation'])

    filtered_df = df[~((df[lang_col] == lang_secondary) & (df['normalized_citation'].isin(primary_citations)))]

    return filtered_df.drop(columns=['normalized_citation'])

In [ ]:
def immigration_cases(text):
    """
    Checks whether the given text contains references to immigration-related ministries 
    within the first 10 lines.

    This function is typically used to filter legal case documents that mention 
    either "Citizenship and Immigration" or "Citoyenneté et Immigration" early in the text.

    Parameters:
    ----------
    text : str or None
        The textual content of a legal case, potentially containing multiple lines.

    Returns:
    -------
    bool
        True if either phrase appears in the first 10 lines of the text; False otherwise.
    """
    if pd.isna(text):
        return False
    lines = text.splitlines()[:10]
    joined_lines = ' '.join(lines)
    return (
        # "Public Safety" in joined_lines or
        # "Immigration, Refugees and Citizenship" in joined_lines or
        "Citizenship and Immigration" in joined_lines or
        "Citoyenneté et Immigration" in joined_lines or
        "MCI" in joined_lines
    )

In [4]:
def filter_refugee_cases(df, text_column="unofficial_text"):
    """
    Removes rows containing refugee exclusion terms from the DataFrame.

    Parameters:
    ----------
    df : pd.DataFrame
    text_column : str

    Returns:
    -------
    pd.DataFrame: Filtered DataFrame without refugee-related documents
    """
    mask = ~df[text_column].str.contains(RE_exclude_refugee, na=False)
    return df[mask].copy()

In [5]:
def filter_inadmissibility(df, text_column="unofficial_text"):
    """
    Filters rows in the DataFrame that contain 'inadmissible' or 'inadmissibility'
    in the relevant numbered lines extracted from the specified text column.

    Parameters:
    df (pd.DataFrame): Input DataFrame
    text_column (str): Name of the column containing case text

    Returns:
    pd.DataFrame: Filtered DataFrame with only relevant cases
    """

    return df[df[text_column].apply(lambda text: 
           'inadmissible' in text.lower() or
           'inadmissibility' in text.lower())]

In [ ]:
def categorize_document(text):
    """
    Extracts relevant numbered lines and classifies a legal document 
    into IRPA inadmissibility grounds.

    Returns:
    - [single ground] if a section is matched.
    - [multiple grounds] if matched by keyword.
    - ['other'] if nothing matches.

    Parameters:
    ----------
    text : str

    Returns:
    -------
    list of str
    """

    # Step 1: Extract relevant numbered lines
    lines = text.splitlines()
    extracted = []
    start_extracting = False

    for line in lines:
        line_strip = line.strip()
        if not start_extracting:
            if "[1]" in line_strip:
                extracted.append(line)
                start_extracting = True
        else:
            if line_strip.startswith("[") and line_strip[1:line_strip.find("]")].isdigit():
                extracted.append(line)
            elif line_strip and line_strip[0].isdigit():
                extracted.append(line)
            else:
                break

    extracted_text = "\n".join(extracted)

    # Step 2: Check section patterns
    for category, pattern in SECTION_PATTERNS.items():
        if re.search(pattern, extracted_text):
            return [category]

    # Step 3: Fallback to keyword patterns
    matched_keywords = [
        category for category, pattern in RE_patterns.items()
        if re.search(pattern, extracted_text)
    ]

    return matched_keywords if matched_keywords else ['other']


In [7]:
RE_exclude_refugee = re.compile(
    r'\b(Refugee Protection Division|convention refugees?|persons? in need of protection|refugee claimants?|protected persons?|réfugiés?)\b',
    re.IGNORECASE
)

SECTION_PATTERNS = {
    'security': re.compile(r'\b(?:s(?:ection)?\.?\s*|subsection|paragraphs?)\s*34\b|\b34\(\d+\)', re.IGNORECASE),
    'human_rights': re.compile(r'\b(?:s(?:ection)?\.?\s*|subsection|paragraphs?)\s*35\b|\b35\(\d+\)', re.IGNORECASE),
    'serious_criminality': re.compile(r'\b(?:s(?:ection)?\.?\s*|subsection|paragraphs?)\s*36\(1\)', re.IGNORECASE),
    'criminality': re.compile(r'\b(?:s(?:ection)?\.?\s*|subsection|paragraphs?)\s*36\(2\)', re.IGNORECASE),
    'organized_criminality': re.compile(r'\b(?:s(?:ection)?\.?\s*|subsection|paragraphs?)\s*37\b|\b37\(\d+\)', re.IGNORECASE),
    'health_grounds': re.compile(r'\b(?:s(?:ection)?\.?\s*|subsection|paragraphs?)\s*38\b|\b38\(\d+\)', re.IGNORECASE),
    'financial_reasons': re.compile(r'\b(?:s(?:ection)?\.?\s*|subsection|paragraphs?)\s*39\b|\b39\(\d+\)', re.IGNORECASE),
    'misrepresentation': re.compile(r'\b(?:s(?:ection)?\.?\s*|subsection|paragraphs?)\s*40\b|\b40\(\d+\)', re.IGNORECASE),
    'non_compliance': re.compile(r'\b(?:s(?:ection)?\.?\s*|subsection|paragraphs?)\s*41\b|\b41\(\d+\)', re.IGNORECASE),
    'inadmissible_family': re.compile(r'\b(?:s(?:ection)?\.?\s*|subsection|paragraphs?)\s*42\b|\b42\(\d+\)', re.IGNORECASE)
}

RE_patterns = {
    'security': re.compile(
        r'\b(espionages?|against canada|canada[’\'‘`s]* interests?|subversions?|democratic governments?|terrorisms?|dangers? to security|violences?|endangerments?|memberships?|complicity|reasonable grounds? to believe)\b',
        re.IGNORECASE
    ),
    'human_rights': re.compile(
        r'\b(human rights?|international rights?|violations?|senior officials?|governments?|regimes?|genocides?|war crimes?|crimes? against humanity|participations?|contributions?|reasonable grounds? to believe|terrorisms?)\b',
        re.IGNORECASE
    ),
    'serious_criminality': re.compile(
        r'\b(criminal convictions?|foreign convictions?|imprisonments?|10 years|ten years|sentences?|over (6|six) months|serious indictable offences?|commissions?|reasonable grounds? to believe)\b',
        re.IGNORECASE
    ),
    'criminality': re.compile(
        r'\b(criminal convictions?|foreign convictions?|indictments?|indictable offences?|summary offences?|commissions?)\b',
        re.IGNORECASE
    ),
    'organized_criminality': re.compile(
        r'\b(memberships?|criminal activities?|organized crimes?|acting in concert|people smuggling|traffickings?|money launderings?|proceeds? of crime|reasonable grounds? to believe)\b',
        re.IGNORECASE
    ),
    'health_grounds': re.compile(
        r'\b(dangers? to public health|dangers? to public safety|excessive demands? on health services|excessive demands? on social services)\b',
        re.IGNORECASE
    ),
    'financial_reasons': re.compile(
        r'\b(unable or unwilling to support (oneself|dependents?)|arrangements? for care and support|social assistances?)\b',
        re.IGNORECASE
    ),
    'misrepresentation': re.compile(
        r'\b(misrepresenting|withholding|material facts?|errors? in administration|non-disclosures?|omissions?|false statements?|false information)\b',
        re.IGNORECASE
    ),
    'non_compliance': re.compile(
        r'\b(contraventions?|non-compliances?|failures? to comply)\b',
        re.IGNORECASE
    ),
    'inadmissible_family': re.compile(
        r'\b(inadmissible family members?|accompanying family members?)\b',
        re.IGNORECASE
    )
}

# Federal Court Data

In [8]:
dataset = load_dataset("refugee-law-lab/canadian-legal-data", "FC", split="train")
FC = dataset.to_pandas()
FC_2014_2024 = FC.query("year >= 2014")
FC_2014_2024

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
18485,2014 FC 1,,FC,2014,Telus Communications Company v. Canada (Attorn...,en,2014-01-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Telus Communications Company v. Canada (Attorn...,
18486,2014 FC 10,,FC,2014,Singh v. Canada (Citizenship and Immigration),en,2014-01-07,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Singh v. Canada (Citizenship and Immigration)\...,
18487,2014 FC 100,,FC,2014,Higgins v. Canada (Attorney General),en,2014-01-29,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Higgins v. Canada (Attorney General)\nCourt (s...,
18488,2014 FC 1000,,FC,2014,Vinat v. Canada (Citizenship and Immigration),en,2014-10-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Vinat v. Canada (Citizenship and Immigration)\...,
18489,2014 FC 1001,,FC,2014,Khadr v. Canada,en,2014-11-04,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Khadr v. Canada\nCourt (s) Database\nFederal C...,
...,...,...,...,...,...,...,...,...,...,...,...
63563,2024 CF 982,,FC,2024,Peralta Rojas c. Canada (Citoyenneté et Immigr...,fr,2024-07-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Peralta Rojas c. Canada (Citoyenneté et Immigr...,
63564,2024 CF 983,,FC,2024,Butt et al v. Canada,fr,2024-06-25,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-09-07,Butt c. Canada\nBase de données – Cour (s)\nDé...,
63565,2024 CF 984,,FC,2024,Sanchez Herrera c. Canada (Sécurité Publique e...,fr,2024-06-25,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Sanchez Herrera c. Canada (Sécurité publique e...,
63566,2024 CF 99,,FC,2024,Nooristani c. Canada (Citoyenneté et Immigration),fr,2024-01-22,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2025-01-03,Nooristani c. Canada (Citoyenneté et Immigrati...,


In [9]:
FC_remove_translation = remove_translated_cases(FC_2014_2024)
FC_remove_translation

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
18485,2014 FC 1,,FC,2014,Telus Communications Company v. Canada (Attorn...,en,2014-01-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Telus Communications Company v. Canada (Attorn...,
18486,2014 FC 10,,FC,2014,Singh v. Canada (Citizenship and Immigration),en,2014-01-07,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Singh v. Canada (Citizenship and Immigration)\...,
18487,2014 FC 100,,FC,2014,Higgins v. Canada (Attorney General),en,2014-01-29,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Higgins v. Canada (Attorney General)\nCourt (s...,
18488,2014 FC 1000,,FC,2014,Vinat v. Canada (Citizenship and Immigration),en,2014-10-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Vinat v. Canada (Citizenship and Immigration)\...,
18489,2014 FC 1001,,FC,2014,Khadr v. Canada,en,2014-11-04,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Khadr v. Canada\nCourt (s) Database\nFederal C...,
...,...,...,...,...,...,...,...,...,...,...,...
63537,2024 CF 88,,FC,2024,Matas c. Canada (Affaires mondiales),fr,2024-01-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-04-10,Matas c. Canada (Affaires mondiales)\nBase de ...,
63554,2024 CF 966,,FC,2024,Quintero c. Canada (Citoyenneté et Immigration),fr,2024-06-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Blandon Quintero c. Canada (Citoyenneté et Imm...,
63563,2024 CF 982,,FC,2024,Peralta Rojas c. Canada (Citoyenneté et Immigr...,fr,2024-07-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Peralta Rojas c. Canada (Citoyenneté et Immigr...,
63565,2024 CF 984,,FC,2024,Sanchez Herrera c. Canada (Sécurité Publique e...,fr,2024-06-25,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Sanchez Herrera c. Canada (Sécurité publique e...,


In [10]:
FC_immigration = FC_remove_translation[FC_remove_translation['unofficial_text'].apply(immigration_cases)]
FC_immigration

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
18486,2014 FC 10,,FC,2014,Singh v. Canada (Citizenship and Immigration),en,2014-01-07,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Singh v. Canada (Citizenship and Immigration)\...,
18488,2014 FC 1000,,FC,2014,Vinat v. Canada (Citizenship and Immigration),en,2014-10-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Vinat v. Canada (Citizenship and Immigration)\...,
18490,2014 FC 1002,,FC,2014,Almrei v. Canada (Citizenship and Immigration),en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Almrei v. Canada (Citizenship and Immigration)...,
18491,2014 FC 1003,,FC,2014,Avagyan v. Canada (Citizenship and Immigration),en,2014-10-22,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Avagyan v. Canada (Citizenship and Immigration...,
18492,2014 FC 1004,,FC,2014,Avagyan v. Canada (Citizenship and Immigration),en,2014-10-22,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Avagyan v. Canada (Citizenship and Immigration...,
...,...,...,...,...,...,...,...,...,...,...,...
63487,2024 CF 677,,FC,2024,Ramirez c. Canada (Citoyenneté et Immigration),fr,2024-05-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Betanzos Ramirez c. Canada (Citoyenneté et Imm...,
63502,2024 CF 74,,FC,2024,Elnour c. Canada (Citoyenneté et Immigration),fr,2024-01-18,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-04-10,Abdallah Elnour El Senoussi c. Canada (Citoyen...,
63515,2024 CF 79,,FC,2024,Callado c. Canada (Citoyenneté et Immigration),fr,2024-01-18,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-04-10,Contreras Callado c. Canada (Citoyenneté et Im...,
63554,2024 CF 966,,FC,2024,Quintero c. Canada (Citoyenneté et Immigration),fr,2024-06-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Blandon Quintero c. Canada (Citoyenneté et Imm...,


In [11]:
FC_non_refugee = filter_refugee_cases(FC_immigration)
FC_non_refugee

/var/folders/f9/98mwc4pn4l51n7xqzs0w7ryh0000gn/T/ipykernel_17031/2680412295.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = ~df[text_column].str.contains(RE_exclude_refugee, na=False)


,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
18500,2014 FC 1012,,FC,2014,Po v. Canada (Citizenship and Immigration),en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Po v. Canada (Citizenship and Immigration)\nCo...,
18502,2014 FC 1015,,FC,2014,Gordon v. Canada (Citizenship and Immigration),en,2014-10-24,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Gordon v. Canada (Citizenship and Immigration)...,
18505,2014 FC 1018,,FC,2014,Sallahi v. Canada (Citizenship and Immigration),en,2014-10-27,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Sallahi v. Canada (Citizenship and Immigration...,
18514,2014 FC 1028,,FC,2014,Afzal v. Canada (Citizenship and Immigration),en,2014-10-29,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Afzal v. Canada (Citizenship and Immigration)\...,
18518,2014 FC 1031,,FC,2014,Sandhu v. Canada (Citizenship and Immigration),en,2014-10-31,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Sandhu v. Canada (Citizenship and Immigration)...,
...,...,...,...,...,...,...,...,...,...,...,...
32856,2024 FC 990,,FC,2024,Dhaliwal v. Canada (Citizenship and Immigration),en,2024-06-25,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Dhaliwal v. Canada (Citizenship and Immigratio...,
32858,2024 FC 992,,FC,2024,Shukurova v. Canada (Citizenship and Immigration),en,2024-06-26,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Shukurova v. Canada (Citizenship and Immigrati...,
32862,2024 FC 998,,FC,2024,Singh Rai v. Canada (Citizenship and Immigrat...,en,2024-06-26,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Singh Rai v. Canada (Citizenship and Immigrati...,
32863,2024 FC 999,,FC,2024,Patel v. Canada (Citizenship and Immigration),en,2024-06-26,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Patel v. Canada (Citizenship and Immigration)\...,


In [12]:
FC_inadmissible = filter_inadmissibility(FC_non_refugee)
FC_inadmissible

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
18532,2014 FC 105,,FC,2014,Muthui v. Canada (Citizenship and Immigration),en,2014-01-30,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Muthui v. Canada (Citizenship and Immigration)...,
18595,2014 FC 112,,FC,2014,Adewole v. Canada (Citizenship and Immigration),en,2014-02-04,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Adewole v. Canada (Citizenship and Immigration...,
18609,2014 FC 1137,,FC,2014,Sidhu v. Canada (Citizenship and Immigration),en,2014-11-25,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Sidhu v. Canada (Citizenship and Immigration)\...,
18617,2014 FC 1146,,FC,2014,Abebe v. Canada (Citizenship and Immigration),en,2014-11-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Abebe v. Canada (Citizenship and Immigration)\...,
18618,2014 FC 1147,,FC,2014,Bundhel v. Canada (Citizenship and Immigration),en,2014-11-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Bundhel v. Canada (Citizenship and Immigration...,
...,...,...,...,...,...,...,...,...,...,...,...
32835,2024 FC 967,,FC,2024,Hamid v. Canada (Citizenship and Immigration),en,2024-06-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Hamid v. Canada (Citizenship and Immigration)\...,
32838,2024 FC 971,,FC,2024,Singh v. Canada (Citizenship and Immigration),en,2024-06-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-09-07,Singh v. Canada (Citizenship and Immigration)\...,
32848,2024 FC 980,,FC,2024,Akinokun v. Canada (Citizenship and Immigration),en,2024-06-24,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Akinokun v. Canada (Citizenship and Immigratio...,
32862,2024 FC 998,,FC,2024,Singh Rai v. Canada (Citizenship and Immigrat...,en,2024-06-26,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Singh Rai v. Canada (Citizenship and Immigrati...,


In [13]:
FC_inadmissible['inadmissibility_ground'] = FC_inadmissible['unofficial_text'].apply(categorize_document)
FC_inadmissible

/var/folders/f9/98mwc4pn4l51n7xqzs0w7ryh0000gn/T/ipykernel_17031/2718995083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FC_inadmissible['inadmissibility_ground'] = FC_inadmissible['unofficial_text'].apply(categorize_document)


,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other,inadmissibility_ground
18532,2014 FC 105,,FC,2014,Muthui v. Canada (Citizenship and Immigration),en,2014-01-30,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Muthui v. Canada (Citizenship and Immigration)...,,[inadmissible_family]
18595,2014 FC 112,,FC,2014,Adewole v. Canada (Citizenship and Immigration),en,2014-02-04,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Adewole v. Canada (Citizenship and Immigration...,,[other]
18609,2014 FC 1137,,FC,2014,Sidhu v. Canada (Citizenship and Immigration),en,2014-11-25,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Sidhu v. Canada (Citizenship and Immigration)\...,,[other]
18617,2014 FC 1146,,FC,2014,Abebe v. Canada (Citizenship and Immigration),en,2014-11-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Abebe v. Canada (Citizenship and Immigration)\...,,[security]
18618,2014 FC 1147,,FC,2014,Bundhel v. Canada (Citizenship and Immigration),en,2014-11-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-23,Bundhel v. Canada (Citizenship and Immigration...,,[other]
...,...,...,...,...,...,...,...,...,...,...,...,...
32835,2024 FC 967,,FC,2024,Hamid v. Canada (Citizenship and Immigration),en,2024-06-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Hamid v. Canada (Citizenship and Immigration)\...,,[security]
32838,2024 FC 971,,FC,2024,Singh v. Canada (Citizenship and Immigration),en,2024-06-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-09-07,Singh v. Canada (Citizenship and Immigration)\...,,[other]
32848,2024 FC 980,,FC,2024,Akinokun v. Canada (Citizenship and Immigration),en,2024-06-24,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Akinokun v. Canada (Citizenship and Immigratio...,,[other]
32862,2024 FC 998,,FC,2024,Singh Rai v. Canada (Citizenship and Immigrat...,en,2024-06-26,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Singh Rai v. Canada (Citizenship and Immigrati...,,[misrepresentation]


In [14]:
FC_inadmissible.to_excel("../data/processed/FC_Regex.xlsx")